<a href="https://colab.research.google.com/github/vkjadon/openai/blob/main/05oai_evaluator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We can load the api key from `.env` file by importing `load_dotenv` method of `dotenv` package. We can install `dotenv` package as

> `pip install python-dotenv`

> `from dotenv import load_dotenv, find_dotenv`

`find_dotenv()` : This function searches up the directory tree from your current working directory until it locates the .env file.

`load_dotenv()` : This function reads the variables from the found .env file and loads them into your Python process's environment variables (specifically, os.environ).

In [ ]:
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

client = OpenAI()

In case you are running the code on Google Colab, add your "OPENAI_API_KEY" key in the `Secrets` of the colab environment and call instantiate the `OpenAI` object as below

In [1]:
!pip install -q faiss-cpu pypdf tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.2/328.2 kB 18.8 MB/s eta 0:00:00


In [2]:
from openai import OpenAI
from google.colab import userdata
client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

In [3]:
from google.colab import files

uploaded = files.upload()
pdf_path = list(uploaded.keys())[0]

Saving download.pdf to download.pdf


In [4]:
from pypdf import PdfReader

reader = PdfReader(pdf_path)

text = ""
for page in reader.pages:
    text += page.extract_text()

print(text[:1000])  # preview


Different types of AI
Estimated time: 10 minutes
Introduction
Artiﬁcial intelligence (AI) encompasses a range of systems designed to mimic, enhance, or exceed human capabilities. AI can be categorized based on its capabilitiesand functionalities. Understanding these types and their capabilities highlights the diverse applications and potential of AI technologies.
Objectives
After completing this reading, you will be able to:
Explain the types of artiﬁcial intelligence based on their functionalities.
Explore the capabilities of each type of artiﬁcial intelligence.
AI types
1. Diagnostic/descriptive AI
Diagnostic or descriptive AI focuses on assessing the correctness of behavior by analyzing historical data to understand what happened and why. This type ofAI is instrumental in identifying patterns and trends, performing comparative analyses, and conducting root cause analyses.
Capabilities:
Scenario planning: Helps in creating different future scenarios based on historical data.
Pattern/

In [5]:
def chunk_text(text, chunk_size=800, overlap=100):
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunks.append(text[start:end])
        start = end - overlap
    return chunks

In [6]:
chunks = chunk_text(text)
print("Total chunks:", len(chunks))

Total chunks: 11


In [7]:
import numpy as np

In [8]:
def get_embeddings(texts):
    response = client.embeddings.create(
        model="text-embedding-3-small",
        input=texts
    )
    return np.array([e.embedding for e in response.data])

In [9]:
chunk_embeddings = get_embeddings(chunks)
print(chunk_embeddings.shape)

(11, 1536)


FAISS (Facebook AI Similarity Search) is an open-source library for efficient similarity search and clustering of dense, high-dimensional vectors. It is used to quickly find items in massive datasets that are "similar" to a given query item, solving performance bottlenecks that traditional databases face when dealing with unstructured data like images or text embeddings.

In [10]:
import faiss

dimension = chunk_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(chunk_embeddings)

print("FAISS index size:", index.ntotal)

FAISS index size: 11


In [11]:
def retrieve_context(query, k=4):
    query_embedding = get_embeddings([query])
    distances, indices = index.search(query_embedding, k)
    return "\n\n".join([chunks[i] for i in indices[0]])

In [12]:
def build_context(query):
    retrieved = retrieve_context(query)
    return "\n\n".join(retrieved)

In [13]:
asked_questions = []
def generate_unique_question(topic, difficulty="medium"):
    context = build_context(topic)

    previous_questions = "\n".join(asked_questions) if asked_questions else "None"

    response = client.responses.create(
        model="gpt-4.1-mini",
        input=[
            {
                "role": "system",
                "content": "You are a university examiner."
            },
            {
                "role": "developer",
                "content": (
                    "Generate ONE new exam question "
                    "that has NOT been asked before in this session. "
                    "Do not repeat or rephrase previous questions."
                )
            },
            {
                "role": "user",
                "content": f"""
Context:
{context}

Previously asked questions:
{previous_questions}

Topic:
{topic}
"""
            }
        ]
    )

    question = response.output_text.strip()
    asked_questions.append(question)

    return question


In [19]:
question = generate_unique_question("Types of AI")
print("QUESTION:", question)

QUESTION: Discuss the differences between Diagnostic/Descriptive AI and Prescriptive AI in terms of their objectives and data utilization. How can these differences influence decision-making processes in a business environment?


In [16]:
def evaluate_answer(question, student_answer, max_marks=10):
    context = build_context(question)

    response = client.responses.create(
        model="gpt-4.1-mini",
        input=[
            {
                "role": "system",
                "content": "You are a strict and fair examiner."
            },
            {
                "role": "developer",
                "content": (
                    "Evaluate the student answer using ONLY the provided context. "
                    "Do NOT use external knowledge.\n\n"
                    "Follow this rubric:\n"
                    "1. Concept correctness\n"
                    "2. Coverage of key points\n"
                    "3. Technical accuracy\n"
                    "4. Clarity of explanation\n\n"
                    f"Assign marks out of {max_marks}. "
                    "Return output in JSON with keys: "
                    "score, strengths, missing_points, feedback."
                )
            },
            {
                "role": "user",
                "content": f"""
Context:
{context}

Question:
{question}

Student Answer:
{student_answer}
"""
            }
        ]
    )

    return response.output_text


In [20]:
student_answer = input("Enter student answer:\n")

Enter student answer:
AI is machine with brain


In [21]:
evaluation = evaluate_answer(question, student_answer)
print("EVALUATION RESULT:\n", evaluation)


EVALUATION RESULT:
 {
  "score": 1,
  "strengths": "The student attempts to relate AI to a machine with a brain, which loosely hints at AI's capability to mimic human-like functions.",
  "missing_points": "The response does not address the differences between Diagnostic/Descriptive AI and Prescriptive AI, their objectives, or how they utilize data. Additionally, it fails to explain how these differences influence decision-making processes in a business context.",
  "feedback": "Your answer is very limited and does not cover the required aspects of the question. You need to clearly explain how Diagnostic/Descriptive AI focuses on analyzing historical data to understand past behaviors and outcomes, while Prescriptive AI focuses on determining the optimal course of action by providing recommendations based on data analysis. Additionally, you should discuss how these differences affect business decisions, with Diagnostic AI helping understand what happened and why, and Prescriptive AI prov